In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 8.4
fig_height = 5.44
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"

  # IPython 7.14 deprecated set_matplotlib_formats from IPython
  try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
  except ImportError:
    # Fall back to deprecated location for older IPython versions
    from IPython.display import set_matplotlib_formats
    
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'RDpcR2l0SHViXFRETS1DQUwtTW9kZS1DaG9pY2U='
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\importlib\\_bootstrap.py": 1741044192.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\importlib\\_bootstrap_external.py": 1741044192.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\zipimport.py": 1741044197.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\codecs.py": 1741044191.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\encodings\\aliases.py": 1741044191.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\encodings\\__init__.py": 1741044191.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\encodings\\utf_8.py": 1741044191.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\encodings\\cp1252.py": 1741044191.0, "C:\\Users\\Pukar.Bhandari\\AppData\\Local\\miniconda3\\envs\\wfrc\\Lib\\abc.py": 1741044191.0, "C:\\Users\\Pukar.Bhand

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import importlib.util
from pathlib import Path
import sys
import os
import requests
import io

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def fetch_github(
    url: str,
    mode: str = "private",
    token_env_var: str = "GITHUB_TOKEN"
) -> requests.Response:
    """
    Fetch content from GitHub repositories.

    Args:
        url: GitHub raw URL (e.g., https://raw.githubusercontent.com/...)
        mode: "public" for public repos, "private" for private repos requiring authentication
        token_env_var: Name of environment variable containing GitHub token (default: GITHUB_TOKEN)

    Returns:
        requests.Response object

    Raises:
        ValueError: If token is missing for private mode or invalid mode
        requests.HTTPError: If request fails
    """
    # Validate mode
    if mode not in ["public", "private"]:
        raise ValueError(f"mode must be 'public' or 'private', got '{mode}'")

    if mode == "public":
        response = requests.get(url, timeout=30)
    else:
        token = os.getenv(token_env_var)
        if not token:
            raise ValueError(
                f"GitHub token not found in environment variable '{token_env_var}'. "
                f"Check your .env file has: {token_env_var}=your_token_here"
            )

        headers = {
            'Authorization': f'token {token}',
            'Accept': 'application/vnd.github.v3.raw'
        }
        response = requests.get(url, headers=headers, timeout=30)

    response.raise_for_status()
    return response

In [4]:
def calculate_categorical_shares(
    df,
    category_col,
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition=None,
    category_mapping=None,
    prefix='share',
    purpose_by_period=['HBW', 'HBO', 'NHB'],  # Only HBC and HBSch get daily
    purpose_daily=['HBC', 'HBSch']  # NEW: Purposes that only get daily (no peak/off-peak)
):
    """
    Calculate weighted shares of categories within groups, ensuring they sum to 1.

    Parameters:
    -----------
    df : DataFrame
        Input dataframe
    category_col : str
        Column containing the categories to calculate shares for
    weight_col : str
        Column containing weights for each observation
    group_cols : list
        Columns to group by (e.g., vehicle ownership, purpose, peak/off-peak)
    filter_condition : str or None
        Optional pandas query string to filter data before calculation
    category_mapping : dict or None
        Optional mapping to rename/recode categories
    prefix : str
        Prefix for output column names
    purpose_by_period : list
        Purposes that should NOT get daily aggregates
    purpose_daily : list
        Purposes that ONLY get daily aggregates (no peak/off-peak breakdown)

    Returns:
    --------
    DataFrame with shares in wide format
    """

    # Apply filter and copy in one step
    if filter_condition:
        df_work = df.query(filter_condition).copy()
    else:
        df_work = df.copy()

    # Apply category mapping if provided
    if category_mapping:
        df_work[category_col] = df_work[category_col].map(category_mapping)
        # Remove unmapped values
        df_work = df_work.dropna(subset=[category_col])

    # Remove any NaN values in key columns
    df_work = df_work.dropna(subset=[category_col, weight_col] + group_cols)

    # Convert Veh_Cat3p to string format for consistency
    df_work['Veh_Cat3p'] = df_work['Veh_Cat3p'].astype(str)
    # Map 3 to 3+ for display
    # df_work['Veh_Cat3p'] = df_work['Veh_Cat3p'].replace('3', '3+')

    # Initialize results dictionary
    results = {}

    # Calculate weighted shares for each group combination (peak/off-peak)
    # BUT skip purpose_daily
    for veh in sorted(df_work['Veh_Cat3p'].unique()):
        for purpose in sorted(df_work['Purp5_text'].unique()):
            # Skip peak/off-peak for daily-only purposes
            if purpose in purpose_daily:
                continue

            for peak in sorted(df_work['PK_OK'].unique()):
                subset = df_work.query(
                    f"Veh_Cat3p == '{veh}' & Purp5_text == '{purpose}' & PK_OK == '{peak}'"
                )

                if len(subset) > 0:
                    # Calculate weighted counts by category
                    weighted_counts = subset.groupby(category_col)[weight_col].sum()
                    total_weight = subset[weight_col].sum()

                    for cat, weighted_count in weighted_counts.items():
                        var_name = f"{prefix}_{cat}_{veh}veh"
                        if var_name not in results:
                            results[var_name] = {}
                        results[var_name][(purpose, peak)] = weighted_count / total_weight

    # Calculate "_all" aggregates for peak/off-peak (excluding daily-only purposes)
    for purpose in sorted(df_work['Purp5_text'].unique()):
        # Skip peak/off-peak for daily-only purposes
        if purpose in purpose_daily:
            continue

        for peak in sorted(df_work['PK_OK'].unique()):
            subset = df_work.query(f"Purp5_text == '{purpose}' & PK_OK == '{peak}'")

            if len(subset) > 0:
                weighted_counts = subset.groupby(category_col)[weight_col].sum()
                total_weight = subset[weight_col].sum()

                for cat, weighted_count in weighted_counts.items():
                    var_name = f"{prefix}_{cat}_all"
                    if var_name not in results:
                        results[var_name] = {}
                    results[var_name][(purpose, peak)] = weighted_count / total_weight

    # Calculate daily aggregates for specified purposes only
    daily_purposes = [p for p in df_work['Purp5_text'].unique()
                     if p not in purpose_by_period]

    for purpose in daily_purposes:
        subset = df_work.query(f"Purp5_text == '{purpose}'")

        if len(subset) > 0:
            # For each vehicle category
            for veh in sorted(df_work['Veh_Cat3p'].unique()):
                veh_subset = subset.query(f"Veh_Cat3p == '{veh}'")
                if len(veh_subset) > 0:
                    weighted_counts = veh_subset.groupby(category_col)[weight_col].sum()
                    total_weight = veh_subset[weight_col].sum()

                    for cat, weighted_count in weighted_counts.items():
                        var_name = f"{prefix}_{cat}_{veh}veh"
                        if var_name not in results:
                            results[var_name] = {}
                        results[var_name][(purpose, 'Daily')] = weighted_count / total_weight

            # For "all" vehicle categories
            weighted_counts = subset.groupby(category_col)[weight_col].sum()
            total_weight = subset[weight_col].sum()

            for cat, weighted_count in weighted_counts.items():
                var_name = f"{prefix}_{cat}_all"
                if var_name not in results:
                    results[var_name] = {}
                results[var_name][(purpose, 'Daily')] = weighted_count / total_weight

    # Convert to DataFrame
    df_result = pd.DataFrame(results).T
    df_result.columns = pd.MultiIndex.from_tuples(df_result.columns)
    df_result = df_result.fillna(0)

    # Sort index
    df_result = df_result.sort_index()

    return df_result

In [5]:
# Verify categorical shares sum to 1.0000
def verify_shares(df_shares, tolerance=0.01):
    """
    Verify that shares sum to approximately 1 within each vehicle category group.

    Now that we have separate rows for each vehicle ownership category (0veh, 1veh, 2veh, 3+veh, all),
    we need to verify that shares sum to 1 within each vehicle category separately.
    """
    print("=== Verifying Share Sums ===\n")

    # Extract vehicle categories from index
    # Assuming format: calib_share_{category}_{veh_category}
    vehicle_categories = df_shares.index.str.extract(r'_(\d\+?veh|all)$')[0].unique()

    print(f"Vehicle categories found: {sorted(vehicle_categories)}\n")

    all_good = True
    issues = []

    # Check each vehicle category separately
    for veh_cat in sorted(vehicle_categories):
        # Filter rows for this vehicle category
        mask = df_shares.index.str.endswith(f'_{veh_cat}')
        df_veh = df_shares[mask]

        print(f"--- Checking {veh_cat} ({len(df_veh)} categories) ---")

        # Check sums for each column within this vehicle category
        veh_issues = []
        for col in df_veh.columns:
            col_sum = df_veh[col].sum()

            if abs(col_sum - 1.0) > tolerance:
                all_good = False
                veh_issues.append((col, col_sum))

        if not veh_issues:
            print(f"  ✅ All shares sum to 1.0 for {veh_cat}")
        else:
            print(f"  ⚠️  {len(veh_issues)} columns don't sum to 1.0 for {veh_cat}:")
            for col, sum_val in veh_issues[:5]:  # Show first 5
                print(f"     {col}: {sum_val:.4f}")
            if len(veh_issues) > 5:
                print(f"     ... and {len(veh_issues) - 5} more")
            issues.extend([(veh_cat, col, sum_val) for col, sum_val in veh_issues])
        print()

    print(f"{'='*60}")
    if all_good:
        print("✅ ALL VEHICLE CATEGORIES: Shares sum to 1.0 (within tolerance)")
    else:
        print(f"⚠️  WARNING: {len(issues)} total column/vehicle combinations don't sum to 1.0")

    print(f"\nTotal columns: {len(df_shares.columns)}")
    print(f"Total vehicle categories: {len(vehicle_categories)}")
    print(f"Total combinations: {len(df_shares.columns) * len(vehicle_categories)}")

    # Return sums by vehicle category
    return {veh_cat: df_shares[df_shares.index.str.endswith(f'_{veh_cat}')].sum(axis=0)
            for veh_cat in sorted(vehicle_categories)}

In [6]:
#| eval: false
#| echo: false

# # Import global TDM functions from local 'Resource' clone

# import sys
# sys.path.insert(0, '../Resources/2-Python/global-functions')
# import BigQuery

# client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()

In [7]:
# # Import global TDM functions from remote 'Resource' repo

# Fetch and import BigQuery module
response = fetch_github(
    'https://raw.githubusercontent.com/WFRCAnalytics/Resources/refs/heads/master/2-Python/global-functions/BigQuery.py',
    mode="public"
)

BigQuery = importlib.util.module_from_spec(importlib.util.spec_from_loader('BigQuery', loader=None))
exec(response.text, BigQuery.__dict__)

# Initiatlize BigQuery Client
client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()

pukar.bhandari
C:/Users/Pukar.Bhandari/.private/confidential-2023-utah-hts-5fd7ddd219a7.json


In [8]:
df_obs_linked = pd.read_csv(
    r"_output/UTA_OBS_2024_Linked_FactorAdjusted_CRT.csv"
)

df_obs_linked

C:\Users\Pukar.Bhandari\AppData\Local\Temp\ipykernel_4828\3457507166.py:1: DtypeWarning:

Columns (8,24,32,37,54,132,174,287) have mixed types. Specify dtype option on import or set low_memory=False.



,ID,DATE_COMPLETED,DATE_TYPE,ROUTE_DIRECTION_Code,ROUTE_DIRECTION,ROUTE_DIRECTION_Other,HOME_ADDRESS_CITY,HOME_ADDRESS_STATE,HOME_ADDRESS_ZIP,HOME_ADDRESS_LAT,...,access_dist,access_time,egress_dist,egress_time,Adj_Factor,trip_weight,uses_crt,valid_crt_mask,dist_CRT,class_CRT
0,5948,2024-02-27,Weekday,UTA_1_2_00,2 200 SOUTH - TO U HOSPITAL,NaN,Clearfield,UT,84015,41.119565,...,0.37,8.79,0.68,16.27,1.195057,7.988529,False,False,NaN,NaN
1,6067,2024-02-27,Weekday,UTA_1_47_00,47 4700 SOUTH - TO W VALLEY CTL,NaN,West Valley City,UT,84119,40.689590,...,0.56,13.53,0.24,5.70,1.407280,4.420834,False,False,NaN,NaN
2,6073,2024-02-28,Weekday,UTA_1_750_01,FRONTRUNNER 750 - SOUTHBOUND,NaN,Ogden,UT,84401,41.236749,...,4.17,8.22,1.37,3.29,1.192511,5.413118,True,True,44.26,CRT_04
3,6077,2024-02-28,Weekday,UTA_1_47_00,47 4700 SOUTH - TO W VALLEY CTL,NaN,West Valley City,UT,84119,40.669687,...,0.25,6.00,0.49,11.88,1.407280,10.487378,False,False,NaN,NaN
4,6078,2024-02-28,Weekday,UTA_1_220_00,220 HIGHLAND DRIVE / 1300 EAST - TO SANDY,NaN,Salt Lake City,UT,84101,40.765122,...,0.27,6.51,0.19,4.55,1.407280,3.287619,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12015,30339,2024-04-26,Weekday,UTA_1_704_01,TRAX GREEN LINE 704 - TO AIRPORT,NaN,Salt Lake City,UT,84116,40.772420,...,0.31,7.53,0.17,4.07,1.405314,8.528545,False,False,NaN,NaN
12016,30340,2024-04-26,Weekday,UTA_1_704_00,TRAX GREEN LINE 704 - TO WEST VALLEY,NaN,Salt Lake City,UT,84116,40.771361,...,0.46,11.00,1.21,29.10,1.405314,16.406476,False,False,NaN,NaN
12017,30341,2024-04-26,Weekday,UTA_1_704_00,TRAX GREEN LINE 704 - TO WEST VALLEY,NaN,Salt Lake City,UT,84116,40.773232,...,0.17,4.07,0.86,20.68,1.405314,8.203238,False,False,NaN,NaN
12018,1000002,2024-02-29,Weekday,UTA_1_701_00,TRAX BLUE LINE 701 - TO DRAPER,NaN,Salt Lake City,UT,84111,40.751607,...,0.33,7.95,0.29,7.02,0.893041,4.139760,False,False,NaN,NaN


In [9]:
# Load Linked Trips data from 2023 Household Travel Survey
df_hts_linked = client.query(
    "SELECT hh_id, PURP7_t, depart_per, model_trip_mode_WFv10, trip_weight FROM " + 'wfrc-modeling-data.prd_tdm_hts_2023.trip_linked'
).to_dataframe()

df_hts_linked

,hh_id,PURP7_t,depart_per,model_trip_mode_WFv10,trip_weight
0,23149927,HBC,AM,bike,0.000000
1,23005950,HBOth,AM,auto-sov,0.000000
2,23008391,HBOth,AM,auto-occ3p,0.000000
3,23008686,HBOth,AM,auto-occ2,115.118280
4,23011482,HBOth,AM,auto-sov,76.022360
...,...,...,...,...,...
232446,23451990,NHBW,PM,auto-sov,0.000000
232447,23458329,NHBW,PM,auto-occ2,0.000000
232448,23489923,NHBW,PM,auto-sov,17.343985
232449,23493042,NHBW,PM,auto-occ2,59.423909


In [10]:
# Load Household data from 2023 Household Travel Survey
df_hts_hh = client.query(
    "SELECT hh_id, num_vehicles_4cat FROM " + 'wfrc-modeling-data.prd_tdm_hts_2023.hh'
).to_dataframe()

df_hts_hh

,hh_id,num_vehicles_4cat
0,23472690,3
1,23307416,3
2,23317145,2
3,23321176,2
4,23345703,2
...,...,...
11178,23364646,2
11179,23046966,1
11180,23043011,1
11181,23085912,3


In [11]:
df_obs_linked[df_obs_linked['Linked_Mode_txt'] == 'CRT']['class_CRT'].value_counts()

class_CRT
CRT_03    799
CRT_02    489
CRT_04    369
CRT_01    309
CRT_00    168
CRT_05     63
CRT_06     47
CRT_07     25
CRT_08      8
Name: count, dtype: int64

In [12]:
# Get CRT subclasses from class_CRT column
crt_categories = sorted(
    df_obs_linked[df_obs_linked['Linked_Mode_txt'] == 'CRT']['class_CRT']
    .dropna()
    .unique()
)

crt_categories

['CRT_00',
 'CRT_01',
 'CRT_02',
 'CRT_03',
 'CRT_04',
 'CRT_05',
 'CRT_06',
 'CRT_07',
 'CRT_08']

In [13]:
# Compute CRT Subclass Proportions by Purpose Only
# This will be used to expand single CRT category into detailed CRT classes

print("\n" + "=" * 80)
print("COMPUTING CRT SUBCLASS PROPORTIONS BY PURPOSE")
print("=" * 80)

# Filter to CRT trips only
df_crt_only = df_obs_linked[
    (df_obs_linked['Linked_Mode_txt'] == 'CRT') &
    (df_obs_linked['class_CRT'].notna())
].copy()

# Compute P(class_CRT_i | CRT, Purpose)
crt_purpose_props = {}

for purpose in sorted(df_crt_only['Purp5_text'].unique()):
    purpose_subset = df_crt_only[df_crt_only['Purp5_text'] == purpose]

    if len(purpose_subset) > 0:
        # Calculate weighted shares by class_CRT
        weighted_counts = purpose_subset.groupby('class_CRT')['trip_weight'].sum()
        total_weight = purpose_subset['trip_weight'].sum()

        purpose_props = {}
        for crt_class, weight in weighted_counts.items():
            purpose_props[crt_class] = weight / total_weight

        crt_purpose_props[purpose] = purpose_props

        # # Display results
        # print(f"\n{purpose}:")
        # for crt_class in sorted(purpose_props.keys()):
        #     print(f"  P({crt_class} | CRT, {purpose}) = {purpose_props[crt_class]:.4f}")

# # Verify proportions sum to 1.0 for each purpose
# print("\n" + "-" * 60)
# print("Verification - sums by purpose:")
# for purpose, props in crt_purpose_props.items():
#     total = sum(props.values())
#     status = "✅" if abs(total - 1.0) < 0.01 else "⚠️"
#     print(f"  {purpose}: {total:.4f} {status}")


COMPUTING CRT SUBCLASS PROPORTIONS BY PURPOSE


In [14]:
purpose_mapping = {
    'HBOth' : 'HBO',
    'NHBNW' : 'NHB',
    'HBW'   : 'HBW',
    'NHBW'  : 'NHB',
    'HBShp' : 'HBO',
    'HBSch' : 'HBSch',
    'HBC'   : 'HBC'
}

df_hts_linked['Purp5_text'] = df_hts_linked['PURP7_t'].map(purpose_mapping)

In [15]:
# Join household data to linked trips data
df_hts_linked = df_hts_linked.merge(
    df_hts_hh.rename(columns={'num_vehicles_4cat': 'Veh_Cat3p'}),
    on='hh_id',
    how='left'  # Keep all trips, even if household data is missing
)

# Rename the vehicle ownership column to match the OBS data
df_hts_linked = df_hts_linked.rename(columns={'num_vehicles_4cat': 'Veh_Cat3p'})

In [16]:
df_hts_linked['PK_OK'] = np.where(
    df_hts_linked["depart_per"].isin(["AM", "PM"]),
    "PK",
    "OK"
)

In [17]:
(
    df_hts_linked[
        (df_hts_linked['model_trip_mode_WFv10'].isin(
            ["auto-sov", "auto-occ2", "auto-occ3p", "walk-to-transit",
             "drive-to-transit", "walk", "bike"]
        )) &
        (df_hts_linked['Purp5_text'] == 'HBW')
    ]
    .assign(motorized_category=lambda x: x['model_trip_mode_WFv10'].map({
        'auto-sov'         : 'motor',
        'auto-occ2'        : 'motor',
        'auto-occ3p'       : 'motor',
        'walk-to-transit'  : 'motor',
        'drive-to-transit' : 'motor',
        'walk'             : 'nonmotor',
        'bike'             : 'nonmotor'
    }))
    .groupby('motorized_category')['trip_weight']
    .sum()
    .pipe(lambda x: (x / x.sum() * 100).round(2))
)

motorized_category
motor       97.03
nonmotor     2.97
Name: trip_weight, dtype: float64

In [18]:
# Identify transit trips in HTS
hts_transit = df_hts_linked[
    df_hts_linked['model_trip_mode_WFv10'].isin(['walk-to-transit', 'drive-to-transit'])
].copy()

# Filter out microtransit
obs_transit = df_obs_linked[df_obs_linked['Linked_Mode_txt'] != 'MT'].copy()

In [19]:
hts_by_access = hts_transit.groupby('model_trip_mode_WFv10')['trip_weight'].sum()
hts_by_access

model_trip_mode_WFv10
drive-to-transit     34844.620753
walk-to-transit     106748.298633
Name: trip_weight, dtype: float64

In [20]:
if 'Ac_Mode2_Model' in obs_transit.columns:
    obs_by_access = obs_transit.groupby('Ac_Mode2_Model')['trip_weight'].sum().sort_index()

obs_by_access

Ac_Mode2_Model
Drive    18682.851823
Walk     77989.858335
Name: trip_weight, dtype: float64

In [21]:
hts_by_purpose = hts_transit.groupby('Purp5_text')['trip_weight'].sum().sort_index()
hts_by_purpose

Purp5_text
HBC       9625.364706
HBO      57055.318544
HBSch     4709.043372
HBW      41885.316622
NHB      28317.876141
Name: trip_weight, dtype: float64

In [22]:
obs_by_purpose = obs_transit.groupby('Purp5_text')['trip_weight'].sum().sort_index()
obs_by_purpose

Purp5_text
HBC      18108.487940
HBO      30554.732806
HBSch     2668.993423
HBW      37103.804529
NHB       8236.691461
Name: trip_weight, dtype: float64

In [23]:
hts_by_veh = hts_transit.groupby('Veh_Cat3p')['trip_weight'].sum().sort_index()
hts_by_veh

Veh_Cat3p
0    41442.964546
1    39084.528933
2    37437.608191
3    23627.817715
Name: trip_weight, dtype: float64

In [24]:
obs_by_veh = obs_transit.groupby('Veh_Cat3p')['trip_weight'].sum().sort_index()
obs_by_veh

Veh_Cat3p
0    28972.297544
1    27888.807780
2    23944.107756
3    15867.497078
Name: trip_weight, dtype: float64

In [25]:
hts_by_period = hts_transit.groupby('PK_OK')['trip_weight'].sum().sort_index()
hts_by_period

PK_OK
OK    75439.954455
PK    66152.964930
Name: trip_weight, dtype: float64

In [26]:
obs_by_period = obs_transit.groupby('PK_OK')['trip_weight'].sum().sort_index()
obs_by_period

PK_OK
OK    56043.250270
PK    40629.459888
Name: trip_weight, dtype: float64

In [27]:
print("\n" + "=" * 80)
print("3. COMPARISON SUMMARY")
print("=" * 80)

hts_total = hts_transit['trip_weight'].sum()
obs_total = obs_transit['trip_weight'].sum()
ratio = obs_total / hts_total if hts_total > 0 else 0

print(f"\nHTS Daily Transit Trips:  {hts_total:>15,.1f}")
print(f"OBS Daily Transit Trips:  {obs_total:>15,.1f}")
print(f"{'─' * 45}")
print(f"Ratio (OBS/HTS):          {ratio:>15.2f}x")
print(f"Difference:               {obs_total - hts_total:>15,.1f}")
print(f"% Difference:             {((obs_total/hts_total - 1) * 100):>15.1f}%")


3. COMPARISON SUMMARY

HTS Daily Transit Trips:        141,592.9
OBS Daily Transit Trips:         96,672.7
─────────────────────────────────────────────
Ratio (OBS/HTS):                     0.68x
Difference:                     -44,920.2
% Difference:                       -31.7%


In [28]:
print("\n" + "=" * 80)
print("4. DETAILED COMPARISON BY PURPOSE & PERIOD")
print("=" * 80)

# Create comparison dataframe
comparison_data = []

for purpose in sorted(df_hts_linked['Purp5_text'].dropna().unique()):
    for period in ['PK', 'OK']:
        hts_val = hts_transit[
            (hts_transit['Purp5_text'] == purpose) &
            (hts_transit['PK_OK'] == period)
        ]['trip_weight'].sum()

        obs_val = obs_transit[
            (obs_transit['Purp5_text'] == purpose) &
            (obs_transit['PK_OK'] == period)
        ]['trip_weight'].sum()

        comparison_data.append({
            'Purpose': purpose,
            'Period': period,
            'HTS_Trips': hts_val,
            'OBS_Trips': obs_val,
            'Ratio_OBS_HTS': obs_val / hts_val if hts_val > 0 else np.nan,
            'Difference': obs_val - hts_val
        })

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values(['Purpose', 'Period'])

print("\n")
print(df_comparison.to_string(index=False, float_format=lambda x: f'{x:,.2f}'))


4. DETAILED COMPARISON BY PURPOSE & PERIOD


Purpose Period  HTS_Trips  OBS_Trips  Ratio_OBS_HTS  Difference
    HBC     OK   5,617.11  10,732.19           1.91    5,115.08
    HBC     PK   4,008.25   7,376.29           1.84    3,368.04
    HBO     OK  33,731.77  20,734.40           0.61  -12,997.37
    HBO     PK  23,323.55   9,820.33           0.42  -13,503.22
  HBSch     OK   1,472.00   1,278.16           0.87     -193.84
  HBSch     PK   3,237.04   1,390.83           0.43   -1,846.21
    HBW     OK  16,938.12  17,569.58           1.04      631.46
    HBW     PK  24,947.20  19,534.23           0.78   -5,412.97
    NHB     OK  17,680.95   5,728.92           0.32  -11,952.04
    NHB     PK  10,636.92   2,507.77           0.24   -8,129.15


In [29]:
# Non-motorized trips from HTS
df_hts_nonmotor = (
    df_hts_linked[df_hts_linked['model_trip_mode_WFv10'].isin(['walk', 'bike'])]
    [['Veh_Cat3p', 'Purp5_text', 'PK_OK', 'model_trip_mode_WFv10', 'trip_weight']]
    .rename(columns={'trip_weight': 'weight', 'model_trip_mode_WFv10': 'mode'})
)

df_hts_nonmotor

,Veh_Cat3p,Purp5_text,PK_OK,mode,weight
0,2,HBC,PK,bike,0.000000
14,2,HBO,PK,walk,0.000000
17,2,HBO,PK,walk,0.000000
28,0,HBO,PK,walk,0.000000
55,3,HBO,PK,walk,301.250493
...,...,...,...,...,...
232391,3,NHB,PK,walk,0.000000
232397,2,NHB,PK,walk,69.250015
232412,2,NHB,PK,walk,0.000000
232422,3,NHB,PK,walk,2.215895


In [30]:
# Auto trips from HTS
df_hts_auto = (
    df_hts_linked[df_hts_linked['model_trip_mode_WFv10'].isin(['auto-sov', 'auto-occ2', 'auto-occ3p'])]
    [['Veh_Cat3p', 'Purp5_text', 'PK_OK', 'model_trip_mode_WFv10', 'trip_weight']]
    .rename(columns={'trip_weight': 'weight', 'model_trip_mode_WFv10': 'mode'})
)

df_hts_auto

,Veh_Cat3p,Purp5_text,PK_OK,mode,weight
1,2,HBO,PK,auto-sov,0.000000
2,3,HBO,PK,auto-occ3p,0.000000
3,3,HBO,PK,auto-occ2,115.118280
4,2,HBO,PK,auto-sov,76.022360
5,2,HBO,PK,auto-occ2,0.000000
...,...,...,...,...,...
232446,1,NHB,PK,auto-sov,0.000000
232447,2,NHB,PK,auto-occ2,0.000000
232448,1,NHB,PK,auto-sov,17.343985
232449,2,NHB,PK,auto-occ2,59.423909


In [31]:
# Transit trips from OBS
df_obs_transit = (
    df_obs_linked[df_obs_linked['Linked_Mode_txt'] != 'MT']
    [['Veh_Cat3p', 'Purp5_text', 'PK_OK', 'Ac_Mode2_Model', 'trip_weight']]
    .assign(mode=lambda x: x['Ac_Mode2_Model'].map({'Walk': 'walk-to-transit', 'Drive': 'drive-to-transit'}))
    .rename(columns={'trip_weight': 'weight'})
    [['Veh_Cat3p', 'Purp5_text', 'PK_OK', 'mode', 'weight']]
)

df_obs_transit

,Veh_Cat3p,Purp5_text,PK_OK,mode,weight
0,0,HBW,PK,walk-to-transit,7.988529
1,0,HBC,OK,walk-to-transit,4.420834
2,2,HBW,OK,drive-to-transit,5.413118
3,0,HBW,OK,walk-to-transit,10.487378
4,1,HBW,OK,walk-to-transit,3.287619
...,...,...,...,...,...
12015,1,HBO,OK,walk-to-transit,8.528545
12016,0,NHB,OK,walk-to-transit,16.406476
12017,1,NHB,OK,walk-to-transit,8.203238
12018,3,HBC,OK,walk-to-transit,4.139760


In [32]:
# 1. All Trips: Motorized vs Non-motorized
print("\n" + "=" * 60)
print("1. ALL TRIPS: MOTORIZED VS NON-MOTORIZED")
print("=" * 60)

df_alltrips_hybrid = pd.concat([
    df_hts_auto.assign(mode_category='motor'),
    df_obs_transit.assign(mode_category='motor'),
    df_hts_nonmotor.assign(mode_category='nonmotor')
], ignore_index=True)

df_alltrips_cat = calculate_categorical_shares(
    df_alltrips_hybrid,
    category_col='mode_category',
    weight_col='weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition=None,
    category_mapping=None,
    prefix='calib_share'
)

print("\nShape:", df_alltrips_cat.shape)
print("\nFirst few rows:")
print(df_alltrips_cat.head(10))
print("\nVerification:")
verify_shares(df_alltrips_cat)


1. ALL TRIPS: MOTORIZED VS NON-MOTORIZED



Shape: (10, 8)

First few rows:
                                HBO                 HBW                 NHB  \
                                 OK        PK        OK        PK        OK   
calib_share_motor_0veh     0.534796  0.708745  0.655120  0.689343  0.587484   
calib_share_motor_1veh     0.914959  0.906437  0.961154  0.952926  0.916496   
calib_share_motor_2veh     0.928844  0.925370  0.964556  0.976711  0.917138   
calib_share_motor_3veh     0.960164  0.962812  0.982527  0.987463  0.934528   
calib_share_motor_all      0.933134  0.933907  0.963239  0.975381  0.919807   
calib_share_nonmotor_0veh  0.465204  0.291255  0.344880  0.310657  0.412516   
calib_share_nonmotor_1veh  0.085041  0.093563  0.038846  0.047074  0.083504   
calib_share_nonmotor_2veh  0.071156  0.074630  0.035444  0.023289  0.082862   
calib_share_nonmotor_3veh  0.039836  0.037188  0.017473  0.012537  0.065472   
calib_share_nonmotor_all   0.066866  0.066093  0.036761  0.024619  0.080193   

                  

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64}

In [33]:
# 2. NON-MOTORIZED TRIPS
print("\n" + "=" * 60)
print("2. NON-MOTORIZED TRIPS")
print("=" * 60)

# non_motorized_mapping = {
#     'walk' : 'walk',
#     'bike' : 'bike'
# }

df_nonmotorized_cat = calculate_categorical_shares(
    df_hts_linked,
    category_col='model_trip_mode_WFv10',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='model_trip_mode_WFv10 in ["walk", "bike"]',
    category_mapping=None,
    prefix='calib_share'
)

print("\nShape:", df_nonmotorized_cat.shape)
print("\nFirst few rows:")
print(df_nonmotorized_cat.head(10))
print("\nVerification:")
verify_shares(df_nonmotorized_cat)


2. NON-MOTORIZED TRIPS

Shape: (10, 8)

First few rows:
                            HBO                 HBW                 NHB  \
                             OK        PK        OK        PK        OK   
calib_share_bike_0veh  0.093370  0.081922  0.126895  0.184947  0.096749   
calib_share_bike_1veh  0.141585  0.107295  0.225268  0.300143  0.100676   
calib_share_bike_2veh  0.104966  0.139940  0.338888  0.382897  0.076727   
calib_share_bike_3veh  0.131393  0.033276  0.110195  0.564415  0.056874   
calib_share_bike_all   0.117384  0.107575  0.215958  0.377379  0.076507   
calib_share_walk_0veh  0.906630  0.918078  0.873105  0.815053  0.903251   
calib_share_walk_1veh  0.858415  0.892705  0.774732  0.699857  0.899324   
calib_share_walk_2veh  0.895034  0.860060  0.661112  0.617103  0.923273   
calib_share_walk_3veh  0.868607  0.966724  0.889805  0.435585  0.943126   
calib_share_walk_all   0.882616  0.892425  0.784042  0.622621  0.923493   

                                      HBC 

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [34]:
# 3. Motorized Trips: Auto vs Transit
print("\n" + "=" * 60)
print("3. MOTORIZED TRIPS: AUTO VS TRANSIT")
print("=" * 60)

df_motorized_hybrid = pd.concat([
    df_hts_auto.assign(mode_category='auto'),
    df_obs_transit.assign(mode_category='transit')
], ignore_index=True)

df_motorized_cat = calculate_categorical_shares(
    df_motorized_hybrid,
    category_col='mode_category',
    weight_col='weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition=None,
    category_mapping=None,
    prefix='calib_share'
)

print("\nShape:", df_motorized_cat.shape)
print("\nFirst few rows:")
print(df_motorized_cat.head(10))
print("\nVerification:")
verify_shares(df_motorized_cat)


3. MOTORIZED TRIPS: AUTO VS TRANSIT



Shape: (10, 8)

First few rows:
                               HBO                 HBW                 NHB  \
                                OK        PK        OK        PK        OK   
calib_share_auto_0veh     0.726251  0.853462  0.729859  0.484913  0.849794   
calib_share_auto_1veh     0.992345  0.993989  0.961769  0.962011  0.996283   
calib_share_auto_2veh     0.997628  0.998085  0.988071  0.989968  0.998815   
calib_share_auto_3veh     0.998221  0.998945  0.993451  0.995569  0.999087   
calib_share_auto_all      0.995092  0.996495  0.981600  0.985147  0.997392   
calib_share_transit_0veh  0.273749  0.146538  0.270141  0.515087  0.150206   
calib_share_transit_1veh  0.007655  0.006011  0.038231  0.037989  0.003717   
calib_share_transit_2veh  0.002372  0.001915  0.011929  0.010032  0.001185   
calib_share_transit_3veh  0.001779  0.001055  0.006549  0.004431  0.000913   
calib_share_transit_all   0.004908  0.003505  0.018400  0.014853  0.002608   

                              

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64}

In [35]:
# 4. Auto Trips: Drive Alone vs Shared Rides
print("\n" + "=" * 60)
print("4. AUTO TRIPS: DRIVE ALONE VS SHARED RIDES")
print("=" * 60)

auto_ride_mapping = {
    'auto-sov'   : 'alone',
    'auto-occ2'  : 'shared',
    'auto-occ3p' : 'shared'
}

df_auto_riders = calculate_categorical_shares(
    df_hts_linked,
    category_col='model_trip_mode_WFv10',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='model_trip_mode_WFv10 in ["auto-sov", "auto-occ2", "auto-occ3p"]',
    category_mapping=auto_ride_mapping,
    prefix='calib_share'
)

print("\nShape:", df_auto_riders.shape)
print("\nFirst few rows:")
print(df_auto_riders.head(10))
print("\nVerification:")
verify_shares(df_auto_riders)


4. AUTO TRIPS: DRIVE ALONE VS SHARED RIDES



Shape: (10, 8)

First few rows:
                              HBO                 HBW                 NHB  \
                               OK        PK        OK        PK        OK   
calib_share_alone_0veh   0.231886  0.134755  0.251145  0.206815  0.214780   
calib_share_alone_1veh   0.450430  0.422546  0.743629  0.767954  0.464611   
calib_share_alone_2veh   0.343353  0.306012  0.849545  0.875657  0.393618   
calib_share_alone_3veh   0.409285  0.362711  0.866645  0.895452  0.471189   
calib_share_alone_all    0.384813  0.343298  0.833347  0.868592  0.434965   
calib_share_shared_0veh  0.768114  0.865245  0.748855  0.793185  0.785220   
calib_share_shared_1veh  0.549570  0.577454  0.256371  0.232046  0.535389   
calib_share_shared_2veh  0.656647  0.693988  0.150455  0.124343  0.606382   
calib_share_shared_3veh  0.590715  0.637289  0.133355  0.104548  0.528811   
calib_share_shared_all   0.615187  0.656702  0.166653  0.131408  0.565035   

                                        HB

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [36]:
# 5. Shared Ride by the Number of People
print("\n" + "=" * 60)
print("5. SHARED RIDER BY NUMBER OF PEOPLE")
print("=" * 60)

shared_ride_mapping = {
    'auto-occ2' : 'sr2',
    'auto-occ3p': 'sr3'
}

df_shared_by_person = calculate_categorical_shares(
    df_hts_linked,
    category_col='model_trip_mode_WFv10',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='model_trip_mode_WFv10 in ["auto-occ2", "auto-occ3p"]',
    category_mapping=shared_ride_mapping,
    prefix='calib_share'
)

print("\nShape:", df_shared_by_person.shape)
print("\nFirst few rows:")
print(df_shared_by_person.head(10))
print("\nVerification:")
verify_shares(df_shared_by_person)


5. SHARED RIDER BY NUMBER OF PEOPLE



Shape: (10, 8)

First few rows:
                           HBO                 HBW                 NHB  \
                            OK        PK        OK        PK        OK   
calib_share_sr2_0veh  0.709382  0.289280  0.731697  0.857568  0.740220   
calib_share_sr2_1veh  0.565882  0.518618  0.726008  0.795799  0.551582   
calib_share_sr2_2veh  0.512200  0.461630  0.792496  0.756507  0.458916   
calib_share_sr2_3veh  0.574960  0.531453  0.784871  0.702538  0.517496   
calib_share_sr2_all   0.543836  0.493188  0.770847  0.747774  0.498240   
calib_share_sr3_0veh  0.290618  0.710720  0.268303  0.142432  0.259780   
calib_share_sr3_1veh  0.434118  0.481382  0.273992  0.204201  0.448418   
calib_share_sr3_2veh  0.487800  0.538370  0.207504  0.243493  0.541084   
calib_share_sr3_3veh  0.425040  0.468547  0.215129  0.297462  0.482504   
calib_share_sr3_all   0.456164  0.506812  0.229153  0.252226  0.501760   

                                   HBSch       HBC  
                         

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBSch  Daily    1.0
 HBC    Daily    1.0
 dtype: float64}

In [37]:
# 6. Transit by Service Type (within linked transit trips)
print("\n" + "=" * 60)
print("6. TRANSIT BY SERVICE TYPE")
print("=" * 60)

# Base service mapping
service_mapping = {
    'LCL': 'local',
    'COR': 'core',
    'EXP': 'express',
    'LRT': 'lrt',
    'CRT': 'crt',
    'BRT': 'brt'
}

# 6. Transit by Service Type
df_transit_by_service = calculate_categorical_shares(
    df_obs_linked,
    category_col='Linked_Mode_txt',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='Linked_Mode_txt != "MT"',
    category_mapping=service_mapping,
    prefix='calib_share'
)

print("\nShape:", df_transit_by_service.shape)
print("\nFirst few rows:")
print(df_transit_by_service.head(10))
print("\nVerification:")
verify_shares(df_transit_by_service)


6. TRANSIT BY SERVICE TYPE



Shape: (25, 8)

First few rows:
                           HBO                 HBW                 NHB  \
                            OK        PK        OK        PK        OK   
calib_share_brt_0veh  0.088861  0.100063  0.043845  0.096335  0.145118   
calib_share_brt_1veh  0.082129  0.057237  0.043697  0.032109  0.151861   
calib_share_brt_2veh  0.045178  0.039916  0.024628  0.051088  0.204102   
calib_share_brt_3veh  0.069742  0.074473  0.041242  0.048555  0.171744   
calib_share_brt_all   0.074924  0.071330  0.038673  0.055927  0.161952   
calib_share_crt_0veh  0.053186  0.051682  0.066407  0.082220  0.035799   
calib_share_crt_1veh  0.114547  0.126383  0.139780  0.204544  0.086570   
calib_share_crt_2veh  0.165676  0.166737  0.194121  0.275120  0.080481   
calib_share_crt_3veh  0.198923  0.240181  0.268523  0.352321  0.057709   
calib_share_crt_all   0.113275  0.121026  0.153034  0.212772  0.060859   

                                     HBC     HBSch  
                         

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [38]:
# 7. Transit by Access Type (Walk vs Drive) (within linked transit trips)
print("\n" + "=" * 60)
print("7. TRANSIT BY ACCESS TYPE")
print("=" * 60)

access_mapping = {
    'Walk': 'walkacc',
    'Drive': 'driveacc'
}

df_transit_by_access = calculate_categorical_shares(
    df_obs_linked,
    category_col='Ac_Mode2_Model',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='Linked_Mode_txt != "MT"',
    category_mapping=access_mapping,
    prefix='calib_share'
)

print("\nShape:", df_transit_by_access.shape)
print("\nFirst few rows:")
print(df_transit_by_access.head(10))
print("\nVerification:")
verify_shares(df_transit_by_access)


7. TRANSIT BY ACCESS TYPE



Shape: (10, 8)

First few rows:
                                HBO                 HBW                 NHB  \
                                 OK        PK        OK        PK        OK   
calib_share_driveacc_0veh  0.042145  0.041288  0.039403  0.047530  0.054343   
calib_share_driveacc_1veh  0.124906  0.150309  0.152275  0.222280  0.046541   
calib_share_driveacc_2veh  0.176413  0.216849  0.279135  0.389120  0.025997   
calib_share_driveacc_3veh  0.266273  0.268424  0.436417  0.433596  0.038132   
calib_share_driveacc_all   0.123080  0.138864  0.197175  0.252116  0.044596   
calib_share_walkacc_0veh   0.957855  0.958712  0.960597  0.952470  0.945657   
calib_share_walkacc_1veh   0.875094  0.849691  0.847725  0.777720  0.953459   
calib_share_walkacc_2veh   0.823587  0.783151  0.720865  0.610880  0.974003   
calib_share_walkacc_3veh   0.733727  0.731576  0.563583  0.566404  0.961868   
calib_share_walkacc_all    0.876920  0.861136  0.802825  0.747884  0.955404   

                  

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [39]:
# 8. Walk-to-Transit by Service Type (within walk-to-transit trips)
print("\n" + "=" * 60)
print("8. WALK-TO-TRANSIT BY SERVICE TYPE")
print("=" * 60)

# Base walk-to-transit mapping
walk_service_mapping = {
    'LCL': 'wlocal',
    'COR': 'wcore',
    'EXP': 'wexpress',
    'LRT': 'wlrt',
    'CRT': 'wcrt',
    'BRT': 'wbrt'
}

# Auto-generate CRT mappings
for crt_cat in crt_categories:
    suffix = '' if crt_cat == 'CRT' else crt_cat.split('_')[1].lower()
    walk_service_mapping[crt_cat] = f'wcrt{suffix}'

# 8. Walk-to-Transit by Service Type
df_walk_to_transit = calculate_categorical_shares(
    df_obs_linked,
    category_col='Linked_Mode_txt',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='Ac_Mode2_Model == "Walk" & Linked_Mode_txt != "MT"',
    category_mapping=walk_service_mapping,
    prefix='calib_share'
)


print("\nShape:", df_walk_to_transit.shape)
print("\nFirst few rows:")
print(df_walk_to_transit.head(10))
print("\nVerification:")
verify_shares(df_walk_to_transit)


8. WALK-TO-TRANSIT BY SERVICE TYPE



Shape: (25, 8)

First few rows:
                            HBO                 HBW                 NHB  \
                             OK        PK        OK        PK        OK   
calib_share_wbrt_0veh  0.092771  0.104372  0.045643  0.101143  0.148794   
calib_share_wbrt_1veh  0.093851  0.067362  0.051547  0.039559  0.159274   
calib_share_wbrt_2veh  0.054856  0.050969  0.034165  0.062345  0.203986   
calib_share_wbrt_3veh  0.091713  0.095129  0.059455  0.085725  0.178553   
calib_share_wbrt_all   0.085061  0.082176  0.046614  0.069424  0.166518   
calib_share_wcrt_0veh  0.040459  0.038498  0.062529  0.069506  0.019720   
calib_share_wcrt_1veh  0.053546  0.054532  0.092601  0.105899  0.071298   
calib_share_wcrt_2veh  0.090631  0.066456  0.100877  0.105374  0.061502   
calib_share_wcrt_3veh  0.052965  0.088973  0.099876  0.203118  0.025995   
calib_share_wcrt_all   0.055574  0.053794  0.085015  0.103989  0.042503   

                                      HBC     HBSch  
            

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [40]:
# 9. Drive-to-Transit by Service Type (within drive-to-transit trips)
print("\n" + "=" * 60)
print("9. DRIVE-TO-TRANSIT BY SERVICE TYPE")
print("=" * 60)

# Base drive-to-transit mapping
drive_service_mapping = {
    'LCL': 'dlocal',
    'COR': 'dcore',
    'EXP': 'dexpress',
    'LRT': 'dlrt',
    'CRT': 'dcrt',
    'BRT': 'dbrt',
}

# 9. Drive-to-Transit by Service Type
df_drive_to_transit = calculate_categorical_shares(
    df_obs_linked,
    category_col='Linked_Mode_txt',
    weight_col='trip_weight',
    group_cols=['Veh_Cat3p', 'Purp5_text', 'PK_OK'],
    filter_condition='Ac_Mode2_Model == "Drive" & Linked_Mode_txt != "MT"',
    category_mapping=drive_service_mapping,
    prefix='calib_share'
)

print("\nShape:", df_drive_to_transit.shape)
print("\nFirst few rows:")
print(df_drive_to_transit.head(10))
print("\nVerification:")
verify_shares(df_drive_to_transit)


9. DRIVE-TO-TRANSIT BY SERVICE TYPE



Shape: (24, 8)

First few rows:
                            HBO                 HBW                 NHB  \
                             OK        PK        OK        PK        OK   
calib_share_dbrt_0veh  0.000000  0.000000  0.000000  0.000000  0.081156   
calib_share_dbrt_1veh  0.000000  0.000000  0.000000  0.006040  0.000000   
calib_share_dbrt_2veh  0.000000  0.000000  0.000000  0.033415  0.208436   
calib_share_dbrt_3veh  0.009200  0.018177  0.017723  0.000000  0.000000   
calib_share_dbrt_all   0.002703  0.004070  0.006341  0.015889  0.064129   
calib_share_dcrt_0veh  0.342430  0.357806  0.160948  0.336995  0.315598   
calib_share_dcrt_1veh  0.541918  0.532558  0.402430  0.549689  0.399439   
calib_share_dcrt_2veh  0.516026  0.528904  0.434922  0.541603  0.791564   
calib_share_dcrt_3veh  0.601117  0.652292  0.486311  0.547224  0.857674   
calib_share_dcrt_all   0.524379  0.537952  0.429982  0.535468  0.454111   

                            HBC       NHB     HBSch  
            

{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64}

In [41]:
def expand_crt_categories(df_shares, crt_purpose_props, crt_var_prefix='crt'):
    """
    Expand single CRT category into detailed CRT subclasses.

    Parameters:
    -----------
    df_shares : DataFrame
        DataFrame with shares including single CRT category
    crt_purpose_props : dict
        Dictionary mapping Purpose -> {class_CRT: proportion}
    crt_var_prefix : str
        Prefix for CRT variable (e.g., 'crt', 'wcrt', 'dcrt')

    Returns:
    --------
    DataFrame with expanded CRT categories
    """
    df_expanded = df_shares.copy()

    # Find all CRT rows (e.g., calib_share_crt_all, calib_share_wcrt_2veh)
    crt_pattern = f'calib_share_{crt_var_prefix}_'
    crt_rows = df_expanded.index[df_expanded.index.str.contains(crt_pattern)].tolist()

    if len(crt_rows) == 0:
        return df_expanded

    # Dictionary to store new rows
    new_rows_dict = {}

    for crt_row in crt_rows:
        # Extract vehicle category from variable name
        # e.g., "calib_share_crt_all" -> "all"
        # e.g., "calib_share_wcrt_2veh" -> "2veh"
        veh_cat = crt_row.split('_')[-1]

        # Get CRT share for each purpose/period combination
        for col in df_expanded.columns:
            purpose, period = col

            crt_share = df_expanded.loc[crt_row, col]

            # Skip if CRT share is 0 or purpose not in props
            if crt_share == 0 or purpose not in crt_purpose_props:
                continue

            # Expand into CRT subclasses
            for crt_class, crt_prop in crt_purpose_props[purpose].items():
                # Create variable name for this CRT subclass
                # e.g., "calib_share_crt00_all", "calib_share_wcrt01_2veh"
                crt_suffix = crt_class.lower().replace('_', '')  # CRT_00 -> crt00
                new_var_name = f"calib_share_{crt_var_prefix}{crt_suffix[3:]}_{veh_cat}"

                # Calculate expanded share
                expanded_share = crt_share * crt_prop

                # Initialize row if it doesn't exist
                if new_var_name not in new_rows_dict:
                    new_rows_dict[new_var_name] = pd.Series(0.0, index=df_expanded.columns, name=new_var_name)

                # Set the value
                new_rows_dict[new_var_name][col] = expanded_share

    # Add new rows and remove old CRT rows
    if new_rows_dict:
        # Remove original single CRT rows first
        df_result = df_expanded.drop(crt_rows)

        # Convert new rows dict to DataFrame and concatenate
        df_new_rows = pd.DataFrame.from_dict(new_rows_dict, orient='index')
        df_result = pd.concat([df_result, df_new_rows], axis=0)

        # Sort index (now all indices should be strings)
        return df_result.sort_index()

    return df_expanded

In [42]:
# Expand CRT in transit by service
print("\nExpanding CRT in Transit by Service...")
df_transit_by_service_expanded = expand_crt_categories(
    df_transit_by_service,
    crt_purpose_props,
    crt_var_prefix='crt'
)
print(f"  Original shape: {df_transit_by_service.shape}")
print(f"  Expanded shape: {df_transit_by_service_expanded.shape}")
print("\nVerification after expansion:")
verify_shares(df_transit_by_service_expanded)


Expanding CRT in Transit by Service...
  Original shape: (25, 8)
  Expanded shape: (65, 8)

Verification after expansion:
=== Verifying Share Sums ===

Vehicle categories found: ['0veh', '1veh', '2veh', '3veh', 'all']

--- Checking 0veh (13 categories) ---
  ✅ All shares sum to 1.0 for 0veh

--- Checking 1veh (13 categories) ---
  ✅ All shares sum to 1.0 for 1veh

--- Checking 2veh (13 categories) ---
  ✅ All shares sum to 1.0 for 2veh

--- Checking 3veh (13 categories) ---
  ✅ All shares sum to 1.0 for 3veh

--- Checking all (13 categories) ---
  ✅ All shares sum to 1.0 for all

✅ ALL VEHICLE CATEGORIES: Shares sum to 1.0 (within tolerance)

Total columns: 8
Total vehicle categories: 5
Total combinations: 40


{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [43]:
# Replace original dataframes with expanded versions
df_transit_by_service = df_transit_by_service_expanded

In [44]:
# Expand CRT in walk-to-transit
print("\nExpanding CRT in Walk-to-Transit...")
df_walk_to_transit_expanded = expand_crt_categories(
    df_walk_to_transit,
    crt_purpose_props,
    crt_var_prefix='wcrt'
)
print(f"  Original shape: {df_walk_to_transit.shape}")
print(f"  Expanded shape: {df_walk_to_transit_expanded.shape}")
print("\nVerification after expansion:")
verify_shares(df_walk_to_transit_expanded)


Expanding CRT in Walk-to-Transit...
  Original shape: (25, 8)
  Expanded shape: (65, 8)

Verification after expansion:
=== Verifying Share Sums ===

Vehicle categories found: ['0veh', '1veh', '2veh', '3veh', 'all']

--- Checking 0veh (13 categories) ---
  ✅ All shares sum to 1.0 for 0veh

--- Checking 1veh (13 categories) ---
  ✅ All shares sum to 1.0 for 1veh

--- Checking 2veh (13 categories) ---
  ✅ All shares sum to 1.0 for 2veh

--- Checking 3veh (13 categories) ---
  ✅ All shares sum to 1.0 for 3veh

--- Checking all (13 categories) ---
  ✅ All shares sum to 1.0 for all

✅ ALL VEHICLE CATEGORIES: Shares sum to 1.0 (within tolerance)

Total columns: 8
Total vehicle categories: 5
Total combinations: 40


{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
        PK       1.0
 HBC    Daily    1.0
 HBSch  Daily    1.0
 dtype: float64}

In [45]:
# Replace original dataframes with expanded versions
df_walk_to_transit = df_walk_to_transit_expanded

In [46]:
# Expand CRT in drive-to-transit
print("\nExpanding CRT in Drive-to-Transit...")
df_drive_to_transit_expanded = expand_crt_categories(
    df_drive_to_transit,
    crt_purpose_props,
    crt_var_prefix='dcrt'
)
print(f"  Original shape: {df_drive_to_transit.shape}")
print(f"  Expanded shape: {df_drive_to_transit_expanded.shape}")
print("\nVerification after expansion:")
verify_shares(df_drive_to_transit_expanded)


Expanding CRT in Drive-to-Transit...
  Original shape: (24, 8)
  Expanded shape: (64, 8)

Verification after expansion:
=== Verifying Share Sums ===

Vehicle categories found: ['0veh', '1veh', '2veh', '3veh', 'all']

--- Checking 0veh (12 categories) ---
  ✅ All shares sum to 1.0 for 0veh

--- Checking 1veh (13 categories) ---
  ✅ All shares sum to 1.0 for 1veh

--- Checking 2veh (13 categories) ---
  ✅ All shares sum to 1.0 for 2veh

--- Checking 3veh (13 categories) ---
  ✅ All shares sum to 1.0 for 3veh

--- Checking all (13 categories) ---
  ✅ All shares sum to 1.0 for all

✅ ALL VEHICLE CATEGORIES: Shares sum to 1.0 (within tolerance)

Total columns: 8
Total vehicle categories: 5
Total combinations: 40


{'0veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '1veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '2veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 '3veh': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64,
 'all': HBO    OK       1.0
        PK       1.0
 HBW    OK       1.0
        PK       1.0
 NHB    OK       1.0
 HBC    Daily    1.0
 NHB    PK       1.0
 HBSch  Daily    1.0
 dtype: float64}

In [47]:
# Replace original dataframes with expanded versions
df_drive_to_transit = df_drive_to_transit_expanded

In [48]:
import pandas as pd
import numpy as np

def create_tree_dataframe(
    df_alltrips_cat,
    df_nonmotorized_cat,
    df_motorized_cat,
    df_auto_riders,
    df_shared_by_person,
    df_transit_by_access,
    df_walk_to_transit,
    df_drive_to_transit,
    purpose='HBW',
    veh_cat='all',
    period=None
):
    """
    Create a hierarchical dataframe using path format.
    Automatically handles any number of CRT classes based on available data.
    """

    def get_value(df, var_name, purpose, veh_cat='all', period=None):
        """Get value for specific purpose, vehicle category, and period"""
        var_full = f"calib_share_{var_name}_{veh_cat}"

        if var_full not in df.index:
            return 0.0

        if period:
            if (purpose, period) in df.columns:
                val = df.loc[var_full, (purpose, period)]
                return val if pd.notna(val) else 0.0
            else:
                return 0.0

        # Otherwise average PK and OK
        total = 0.0
        count = 0

        for per in ['PK', 'OK']:
            if (purpose, per) in df.columns:
                val = df.loc[var_full, (purpose, per)]
                if pd.notna(val):
                    total += val
                    count += 1

        # If no PK/OK, try Daily
        if count == 0 and (purpose, 'Daily') in df.columns:
            val = df.loc[var_full, (purpose, 'Daily')]
            if pd.notna(val):
                return val

        return total / count if count > 0 else 0.0

    # Get all conditional shares (non-transit)
    nonmotor_share = get_value(df_alltrips_cat, 'nonmotor', purpose, veh_cat, period)
    motor_share = 1.0 - nonmotor_share

    walk_share = get_value(df_nonmotorized_cat, 'walk', purpose, veh_cat, period)
    bike_share = get_value(df_nonmotorized_cat, 'bike', purpose, veh_cat, period)

    auto_share = get_value(df_motorized_cat, 'auto', purpose, veh_cat, period)
    transit_share = get_value(df_motorized_cat, 'transit', purpose, veh_cat, period)

    alone_share = get_value(df_auto_riders, 'alone', purpose, veh_cat, period)
    shared_share = get_value(df_auto_riders, 'shared', purpose, veh_cat, period)

    sr2_share = get_value(df_shared_by_person, 'sr2', purpose, veh_cat, period)
    sr3_share = get_value(df_shared_by_person, 'sr3', purpose, veh_cat, period)

    walkacc_share = get_value(df_transit_by_access, 'walkacc', purpose, veh_cat, period)
    driveacc_share = get_value(df_transit_by_access, 'driveacc', purpose, veh_cat, period)

    # Base values
    total_value = 1.0
    display_period = period if period else 'Daily'

    rows = []

    # ========================================================================
    # NON-MOTORIZED BRANCH
    # ========================================================================
    rows.append({
        'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
        'level1': 'Non-motorized', 'level2': 'Walk', 'level3': None, 'level4': None,
        'value': total_value * nonmotor_share * walk_share * 100
    })

    rows.append({
        'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
        'level1': 'Non-motorized', 'level2': 'Bike', 'level3': None, 'level4': None,
        'value': total_value * nonmotor_share * bike_share * 100
    })

    # ========================================================================
    # MOTORIZED > AUTO BRANCH
    # ========================================================================
    rows.append({
        'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
        'level1': 'Motorized', 'level2': 'Auto', 'level3': 'Drive Alone', 'level4': None,
        'value': total_value * motor_share * auto_share * alone_share * 100
    })

    rows.append({
        'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
        'level1': 'Motorized', 'level2': 'Auto', 'level3': 'Shared Ride', 'level4': 'SR2 (2 people)',
        'value': total_value * motor_share * auto_share * shared_share * sr2_share * 100
    })

    rows.append({
        'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
        'level1': 'Motorized', 'level2': 'Auto', 'level3': 'Shared Ride', 'level4': 'SR3 (3+ people)',
        'value': total_value * motor_share * auto_share * shared_share * sr3_share * 100
    })

    # ========================================================================
    # MOTORIZED > TRANSIT > WALK-TO-TRANSIT
    # ========================================================================
    walk_transit_base = total_value * motor_share * transit_share * walkacc_share * 100

    # Get all walk-to-transit variables from the expanded dataframe
    walk_vars = [idx for idx in df_walk_to_transit.index if idx.startswith('calib_share_w')]

    for walk_var in walk_vars:
        # Extract service name from variable (e.g., calib_share_wlrt_all -> LRT)
        parts = walk_var.replace('calib_share_w', '').split('_')
        service_name = parts[0].upper()  # e.g., 'lrt' -> 'LRT', 'crt00' -> 'CRT00'

        if service_name.startswith('CRT'):
            # Format CRT subclasses as CRT_00, CRT_01, etc.
            if len(service_name) > 3:
                service_name = f"CRT_{service_name[3:]}"

        service_share = get_value(df_walk_to_transit, walk_var.replace('calib_share_', '').rsplit('_', 1)[0], purpose, veh_cat, period)

        rows.append({
            'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
            'level1': 'Motorized', 'level2': 'Transit', 'level3': 'Walk-to-Transit', 'level4': service_name,
            'value': walk_transit_base * service_share
        })

    # ========================================================================
    # MOTORIZED > TRANSIT > DRIVE-TO-TRANSIT
    # ========================================================================
    drive_transit_base = total_value * motor_share * transit_share * driveacc_share * 100

    # Get all drive-to-transit variables from the expanded dataframe
    drive_vars = [idx for idx in df_drive_to_transit.index if idx.startswith('calib_share_d')]

    for drive_var in drive_vars:
        # Extract service name from variable (e.g., calib_share_dlrt_all -> LRT)
        parts = drive_var.replace('calib_share_d', '').split('_')
        service_name = parts[0].upper()  # e.g., 'lrt' -> 'LRT', 'crt00' -> 'CRT00'

        if service_name.startswith('CRT'):
            # Format CRT subclasses as CRT_00, CRT_01, etc.
            if len(service_name) > 3:
                service_name = f"CRT_{service_name[3:]}"

        service_share = get_value(df_drive_to_transit, drive_var.replace('calib_share_', '').rsplit('_', 1)[0], purpose, veh_cat, period)

        rows.append({
            'purpose': purpose, 'period': display_period, 'veh_cat': veh_cat,
            'level1': 'Motorized', 'level2': 'Transit', 'level3': 'Drive-to-Transit', 'level4': service_name,
            'value': drive_transit_base * service_share
        })

    return pd.DataFrame(rows)

In [49]:
# Create comprehensive dataframe with all combinations

# Define all categories
purposes = ['HBW', 'HBO', 'NHB', 'HBC', 'HBSch']
periods = ['PK', 'OK', 'Daily']
veh_cats = ['all', '0veh', '1veh', '2veh', '3veh']

# Create list to store all dataframes
all_dfs = []

# Loop through all combinations
for purpose in purposes:
    for period in periods:
        # Skip Daily for purposes that have PK/OK breakdown
        if purpose in ['HBW', 'HBO', 'NHB'] and period == 'Daily':
            continue

        # Skip PK/OK for purposes that only have Daily
        if purpose in ['HBC', 'HBSch'] and period in ['PK', 'OK']:
            continue

        for veh_cat in veh_cats:
            df_temp = create_tree_dataframe(
                df_alltrips_cat,
                df_nonmotorized_cat,
                df_motorized_cat,
                df_auto_riders,
                df_shared_by_person,
                df_transit_by_access,
                df_walk_to_transit,
                df_drive_to_transit,
                purpose=purpose,
                veh_cat=veh_cat,
                period=period
            )
            all_dfs.append(df_temp)

# Combine all dataframes
df_tree_all = pd.concat(all_dfs, ignore_index=True)

df_tree_all.head(20)

,purpose,period,veh_cat,level1,level2,level3,level4,value
0,HBW,PK,all,Non-motorized,Walk,None,None,1.532852
1,HBW,PK,all,Non-motorized,Bike,None,None,0.929083
2,HBW,PK,all,Motorized,Auto,Drive Alone,None,83.462463
3,HBW,PK,all,Motorized,Auto,Shared Ride,SR2 (2 people),9.442074
4,HBW,PK,all,Motorized,Auto,Shared Ride,SR3 (3+ people),3.184839
5,HBW,PK,all,Motorized,Transit,Walk-to-Transit,BRT,0.075218
6,HBW,PK,all,Motorized,Transit,Walk-to-Transit,BRT,0.075218
7,HBW,PK,all,Motorized,Transit,Walk-to-Transit,BRT,0.075218
8,HBW,PK,all,Motorized,Transit,Walk-to-Transit,BRT,0.075218
9,HBW,PK,all,Motorized,Transit,Walk-to-Transit,BRT,0.075218


In [50]:
#| echo: false
#| eval: false

# Create a simple treemap for HBW, all vehicles, PK period
df_filtered = df_tree_all.query("purpose == 'HBW' & veh_cat == 'all' & period == 'PK'").copy()

# Create treemap using path parameter
import plotly.express as px

fig = px.treemap(
    df_filtered,
    path=[px.Constant("All Trips"), 'level1', 'level2', 'level3', 'level4'],
    values='value',
    branchvalues="total"
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [51]:
# Create interactive treemap with percentages of parent and total
import plotly.graph_objects as go
import pandas as pd

# Get unique values for dropdowns
purposes = sorted(df_tree_all['purpose'].unique())
periods = sorted(df_tree_all['period'].unique())
veh_cats = sorted(df_tree_all['veh_cat'].unique())

# Create all trace combinations and store them
traces = []
trace_lookup = {}  # Map (purpose, period, veh_cat) to trace index

for purpose in purposes:
    for period in periods:
        for veh_cat in veh_cats:
            df_filtered = df_tree_all.query(
                f"purpose == '{purpose}' & veh_cat == '{veh_cat}' & period == '{period}'"
            ).copy()

            if len(df_filtered) > 0:
                # Create treemap trace
                labels = []
                parents = []
                values = []
                text = []  # Custom text for hover

                # Root
                labels.append("All Trips")
                parents.append("")
                values.append(100)
                text.append("All Trips<br>100%")

                # Add all hierarchy levels
                for _, row in df_filtered.iterrows():
                    # Level 1
                    if row['level1'] not in labels:
                        level1_value = df_filtered[df_filtered['level1'] == row['level1']]['value'].sum()
                        labels.append(row['level1'])
                        parents.append("All Trips")
                        values.append(level1_value)
                        text.append(f"{row['level1']}<br>{level1_value:.2f}% of total")

                    # Level 2
                    if pd.notna(row['level2']):
                        level2_id = row['level2']
                        if level2_id not in labels:
                            level2_value = df_filtered[
                                (df_filtered['level1'] == row['level1']) &
                                (df_filtered['level2'] == row['level2'])
                            ]['value'].sum()
                            level1_value = df_filtered[df_filtered['level1'] == row['level1']]['value'].sum()
                            pct_of_parent = (level2_value / level1_value * 100) if level1_value > 0 else 0

                            labels.append(level2_id)
                            parents.append(row['level1'])
                            values.append(level2_value)
                            text.append(f"{level2_id}<br>{pct_of_parent:.1f}% of {row['level1']}<br>{level2_value:.2f}% of total")

                        # Level 3
                        if pd.notna(row['level3']):
                            level3_id = row['level3']
                            if level3_id not in labels:
                                level3_value = df_filtered[
                                    (df_filtered['level1'] == row['level1']) &
                                    (df_filtered['level2'] == row['level2']) &
                                    (df_filtered['level3'] == row['level3'])
                                ]['value'].sum()
                                level2_value = df_filtered[
                                    (df_filtered['level1'] == row['level1']) &
                                    (df_filtered['level2'] == row['level2'])
                                ]['value'].sum()
                                pct_of_parent = (level3_value / level2_value * 100) if level2_value > 0 else 0

                                labels.append(level3_id)
                                parents.append(level2_id)
                                values.append(level3_value)
                                text.append(f"{level3_id}<br>{pct_of_parent:.1f}% of {level2_id}<br>{level3_value:.2f}% of total")

                            # Level 4
                            if pd.notna(row['level4']):
                                level4_id = row['level4']
                                level4_value = row['value']
                                level3_value = df_filtered[
                                    (df_filtered['level1'] == row['level1']) &
                                    (df_filtered['level2'] == row['level2']) &
                                    (df_filtered['level3'] == row['level3'])
                                ]['value'].sum()
                                pct_of_parent = (level4_value / level3_value * 100) if level3_value > 0 else 0

                                labels.append(level4_id)
                                parents.append(level3_id)
                                values.append(level4_value)
                                text.append(f"{level4_id}<br>{pct_of_parent:.1f}% of {level3_id}<br>{level4_value:.2f}% of total")

                trace = go.Treemap(
                    labels=labels,
                    parents=parents,
                    values=values,
                    customdata=text,
                    textinfo="label",
                    textposition="middle center",
                    hovertemplate='<b>%{label}</b><br>%{customdata}<extra></extra>',
                    branchvalues="total",
                    visible=False,
                    name=f"{purpose}_{period}_{veh_cat}"
                )

                trace_lookup[(purpose, period, veh_cat)] = len(traces)
                traces.append(trace)

# Set initial trace visible
initial_purpose = 'HBW'
initial_period = 'PK'
initial_veh = 'all'

if (initial_purpose, initial_period, initial_veh) in trace_lookup:
    traces[trace_lookup[(initial_purpose, initial_period, initial_veh)]].visible = True

# Create figure
fig = go.Figure(data=traces)

# Create combined dropdown options (all combinations)
dropdown_options = []

for purpose in purposes:
    for period in periods:
        # Check if this purpose-period combination exists
        has_data = False
        for veh_cat in veh_cats:
            if (purpose, period, veh_cat) in trace_lookup:
                has_data = True
                break

        if not has_data:
            continue

        for veh_cat in veh_cats:
            if (purpose, period, veh_cat) in trace_lookup:
                visible = [False] * len(traces)
                idx = trace_lookup[(purpose, period, veh_cat)]
                visible[idx] = True

                dropdown_options.append(
                    dict(
                        label=f"{purpose} | {period} | {veh_cat}",
                        method="update",
                        args=[
                            {"visible": visible},
                            {"title": f"Mode Choice Hierarchy: {purpose} - {period} - {veh_cat}"}
                        ]
                    )
                )

# Update layout with single combined dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_options,
            direction="down",
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top",
            bgcolor="white",
            bordercolor="gray",
            borderwidth=1
        )
    ],
    title=f"Mode Choice Hierarchy: {initial_purpose} - {initial_period} - {initial_veh}",
    margin=dict(t=120, l=25, r=25, b=25),
    height=800
)

fig.update_traces(root_color="lightgrey")

fig.show()

In [52]:
# Create output directory if it doesn't exist
output_dir = Path("_output")
output_dir.mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("EXPORTING CALIBRATION TARGET FILES")
print("=" * 80)

EXPORTING CALIBRATION TARGET FILES


In [53]:
def extract_values(df, purpose, period, pattern, include_veh_categories=True):
    """
    Extract values from dataframe for given purpose/period matching a pattern.

    Parameters:
    -----------
    df : DataFrame
        DataFrame with MultiIndex columns (purpose, period)
    purpose : str
        Purpose to filter (e.g., 'HBW', 'HBO')
    period : str
        Period to filter (e.g., 'PK', 'OK', 'Daily')
    pattern : str
        Pattern to match in index (e.g., 'motor', 'bike', 'transit')
    include_veh_categories : bool
        If True, include all vehicle categories. If False, only include '_all' and strip suffix.

    Returns:
    --------
    dict : {variable_name: value}
    """
    results = {}

    # Check if column exists
    if (purpose, period) not in df.columns:
        return results

    # Filter rows matching pattern
    mask = df.index.str.contains(pattern)

    for idx in df[mask].index:
        value = df.loc[idx, (purpose, period)]

        if include_veh_categories:
            # Include all vehicle categories as-is
            results[idx] = value
        else:
            # Only include '_all' suffix and strip it
            if idx.endswith('_all'):
                # Remove '_all' suffix
                var_name = idx.rsplit('_', 1)[0]
                results[var_name] = value

    return results

In [54]:
def export_calib_file(purpose, period, include_veh_categories=True):
    """
    Export calibration target file for a given purpose and period.

    Parameters:
    -----------
    purpose : str
        Trip purpose (HBW, HBO, NHB, HBC, HBSch)
    period : str
        Time period (PK, OK, Daily)
    include_veh_categories : bool
        If True, include vehicle category breakdowns (_0veh, _1veh, etc.)
        If False, only use aggregate values without vehicle suffix
    """

    # Determine file period (Daily becomes 'pk' for HBC/HBSch)
    file_period = 'pk' if period == 'Daily' else period.lower()

    # Determine display period for header
    display_period = file_period.capitalize()
    # display_period = 'Pk' if period == 'PK' or period == 'Daily' else 'Ok'

    filename = f"calib_target_{purpose.lower()}_{file_period}.txt"
    filepath = os.path.join(output_dir, filename)

    print(f"\nCreating {filename}... (Vehicle categories: {'Yes' if include_veh_categories else 'No'})")

    with open(filepath, 'w') as f:
        # Write header
        f.write(f";Calibration target values - {purpose} {display_period}\t\t\n")
        f.write("\t\t\n")

        # Write disclaimer
        f.write(f";Note: Not all of the variables listed below are used directly in the Mode Choice model. Some are included solely for verification and cross-checking purposes.\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 1: Motorized vs Non-motorized (within total trips)
        # =====================================================================
        f.write(";relative within total trips\t\t\n")

        # Non-motorized shares only
        nonmotorized = extract_values(df_alltrips_cat, purpose, period, 'nonmotor_', include_veh_categories)
        if nonmotorized:
            f.write("  ;nonmotorized shares \t\t\n")
            for var, val in sorted(nonmotorized.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 2: Walk vs Bike (within nonmotorized)
        # =====================================================================
        f.write(";relative within nonmotorized\t\t\n")

        # Walk shares
        walk = extract_values(df_nonmotorized_cat, purpose, period, 'walk_', include_veh_categories)
        if walk:
            f.write("  ;walk shares \t\t\n")
            for var, val in sorted(walk.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("\t\t\n")

        # Bike shares
        bike = extract_values(df_nonmotorized_cat, purpose, period, 'bike_', include_veh_categories)
        if bike:
            f.write("  ;bike shares \t\t\n")
            for var, val in sorted(bike.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 3: Drive Alone vs Shared Ride (within auto)
        # =====================================================================
        f.write(";relative within auto\t\t\n")

        # Drive alone shares
        alone = extract_values(df_auto_riders, purpose, period, 'alone_', include_veh_categories)
        if alone:
            f.write("  ;drive alone shares \t\t\n")
            for var, val in sorted(alone.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("\t\t\n")

        # Shared ride shares
        shared = extract_values(df_auto_riders, purpose, period, 'shared_', include_veh_categories)
        if shared:
            f.write("  ;shared ride shares \t\t\n")
            for var, val in sorted(shared.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 4: SR2 vs SR3 (within shared ride)
        # =====================================================================
        f.write(";relative within shared ride\t\t\n")

        # SR2 shares
        sr2 = extract_values(df_shared_by_person, purpose, period, 'sr2_', include_veh_categories)
        if sr2:
            f.write("  ;sr2 shares \t\t\n")
            for var, val in sorted(sr2.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("\t\t\n")

        # SR3 shares
        sr3 = extract_values(df_shared_by_person, purpose, period, 'sr3_', include_veh_categories)
        if sr3:
            f.write("  ;sr3 shares \t\t\n")
            for var, val in sorted(sr3.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 5: Auto vs Transit (within motorized)
        # =====================================================================
        f.write(";relative within motorized\t\t\n")

        # Auto shares
        auto = extract_values(df_motorized_cat, purpose, period, 'auto_', include_veh_categories)
        if auto:
            f.write("  ;auto shares \t\t\n")
            for var, val in sorted(auto.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("\t\t\n")

        # Transit shares
        transit = extract_values(df_motorized_cat, purpose, period, 'transit_', include_veh_categories)
        if transit:
            f.write("  ;transit shares \t\t\n")
            for var, val in sorted(transit.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write(" \t\t\n")

        f.write("\t\t\n")

        # =====================================================================
        # SECTION 6: Transit by Service Type (within transit)
        # NOTE: This section now includes expanded CRT subclasses (crt00, crt01, etc.)
        # =====================================================================
        f.write(";relative within transit\t\t\n")

        # Get all transit service types
        service_types = service_mapping.values()
        transit_services = {}
        for service in service_types:
            vals = extract_values(df_transit_by_service, purpose, period, f'{service}_', include_veh_categories)
            transit_services.update(vals)

        if transit_services:
            for var, val in sorted(transit_services.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("  \t\t\n")

        f.write("\t\t\n")

        # =====================================================================
        # SECTION 7: Walk vs Drive Access (within transit)
        # =====================================================================
        f.write(";relative within transit\t\t\n")

        # Walk access shares
        walkacc = extract_values(df_transit_by_access, purpose, period, 'walkacc_', include_veh_categories)
        if walkacc:
            f.write("  ;walk-to-transit shares \t\t\n")
            for var, val in sorted(walkacc.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")
            f.write("\t\t\n")

        # Drive access shares
        driveacc = extract_values(df_transit_by_access, purpose, period, 'driveacc_', include_veh_categories)
        if driveacc:
            f.write("  ;drive-to-transit shares \t\t\n")
            for var, val in sorted(driveacc.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 8: Walk-to-Transit by Service (within walkacc)
        # NOTE: This section now includes expanded CRT subclasses (wcrt00, wcrt01, etc.)
        # =====================================================================
        f.write(";relative within walkacc\t\t\n")

        # Get all walk-to-transit service types
        walk_services = {}
        for service in walk_service_mapping.values():
            vals = extract_values(df_walk_to_transit, purpose, period, f'{service}_', include_veh_categories)
            walk_services.update(vals)

        if walk_services:
            for var, val in sorted(walk_services.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

        f.write("\t\t\n")
        f.write("\t\t\n")

        # =====================================================================
        # SECTION 9: Drive-to-Transit by Service (within driveacc)
        # NOTE: This section now includes expanded CRT subclasses (dcrt00, dcrt01, etc.)
        # =====================================================================
        f.write(";relative within driveacc\t\t\n")

        # Get all drive-to-transit service types
        drive_services = {}
        for service in drive_service_mapping.values():
            vals = extract_values(df_drive_to_transit, purpose, period, f'{service}_', include_veh_categories)
            drive_services.update(vals)

        if drive_services:
            for var, val in sorted(drive_services.items()):
                f.write(f"  {var:<30}\t=\t{val:.4f}\n")

    print(f"  ✅ Written to {filepath}")

In [55]:
# Define purposes and their configuration
export_config = {
    'HBW':   {'periods': ['PK', 'OK'], 'include_veh': True},
    'HBO':   {'periods': ['PK', 'OK'], 'include_veh': True},
    'NHB':   {'periods': ['PK', 'OK'], 'include_veh': True}, # False
    'HBC':   {'periods': ['Daily'],    'include_veh': True}, # False
    'HBSch': {'periods': ['Daily'],    'include_veh': True}  # False
}

for purpose, config in export_config.items():
    for period in config['periods']:
        export_calib_file(purpose, period, include_veh_categories=config['include_veh'])

print("\n" + "=" * 80)
print("✅ EXPORT COMPLETE!")
print("=" * 80)


Creating calib_target_hbw_pk.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbw_pk.txt

Creating calib_target_hbw_ok.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbw_ok.txt

Creating calib_target_hbo_pk.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbo_pk.txt

Creating calib_target_hbo_ok.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbo_ok.txt

Creating calib_target_nhb_pk.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_nhb_pk.txt

Creating calib_target_nhb_ok.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_nhb_ok.txt

Creating calib_target_hbc_pk.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbc_pk.txt

Creating calib_target_hbsch_pk.txt... (Vehicle categories: Yes)
  ✅ Written to _output\calib_target_hbsch_pk.txt

✅ EXPORT COMPLETE!
